### Import

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

os.getcwd()

'/home/rossleecooloh/workspace/Tbrain_credit_card_competition'

### Rolloing 函數

In [2]:
# a = np.array([1,2,3,4,5,6,7,8])
# print(a.cumsum())
# b = a.cumsum()
# print(b[3:])
# print(b[:-3])
# print(b[3:] - b[:-3])
# b[3:] = b[3:] - b[:-3]
# b

a = np.array([1,2,3,4,5,6,7,8])

def rolling_window_sum(x, window_size=3):
    cum_x = x.cumsum()
    cum_x[window_size:] = cum_x[window_size:] - cum_x[:-window_size]
    return cum_x

print(rolling_window_sum(a, 3))

def cumsum_mean(x):
    return x.cumsum() / np.arange(1, len(x) + 1)

print(cumsum_mean(a))

def rolling_window_mean(x, window_size=3):
    cum_x = x.cumsum()
    cum_x[window_size:] = cum_x[window_size:] - cum_x[:-window_size]
    return cum_x / np.arange(1, len(x) + 1)

print(rolling_window_mean(a, 4))

[ 1  3  6  9 12 15 18 21]
[1.  1.5 2.  2.5 3.  3.5 4.  4.5]
[1.         1.5        2.         2.5        2.8        3.
 3.14285714 3.25      ]


In [3]:
a = pd.Series(np.array([1,2,3,4,7,6,7,8]))
# a.rolling(3, min_periods=3).apply(lambda x: min((x[1] - x[0]), (x[2] - x[1])))
# a.rolling(2, min_periods=2).apply(lambda x: x[1] - x[0])
a.rolling(2, min_periods=2).apply(lambda x: x[0] - x[1])

/home/rossleecooloh/.pyenv/versions/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.


0    NaN
1   -1.0
2   -1.0
3   -1.0
4   -3.0
5    1.0
6   -1.0
7   -1.0
dtype: float64

In [12]:
train_pre = pd.read_csv("pre_train_11_no_acc_time_rm_dup.csv")
test_pre = pd.read_csv("pre_test_11_no_acc_time_rm_dup.csv")

print(train_pre.shape)
print(test_pre.shape)

all_features = pd.concat([train_pre.drop("fraud_ind", axis=1), test_pre])
all_features.head(3)

(1516988, 59)
(421665, 58)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,stscd_value_counts,ovrlt_value_counts,flbmk_value_counts,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1915002,1913637,1932259,1799525,223907,1854121,1,1,0,0
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1915002,1913637,1932259,67172,223907,1854121,1,0,0,0
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1915002,1913637,1932259,1799525,223907,1854121,1,1,0,0


### Categorical columns
- bacno 歸戶帳號 (不刪)
- cano 交易卡號 (不刪)
- mchno 特店代號
- acqic 收單行代碼
- contp 交易類別
- etymd 交易型態
- mcc MCC_CODE
- ecfg 網路交易註記
- insfg 分期交易註記
- stocn 消費地國別
- scity 消費城市 --> 考慮刪除?
- stscd 狀態碼
- ovrlt 超額註記碼
- flbmk Fallback 註記
- hcefg 支付形態
- csmcu 消費地幣別
- flg_3dsmk 3DS 交易註記
### Numeric columns
- locdt 授權日期 --> 當類別看，mean encode完刪掉
- loctm 授權時間
- conam 交易金額-台幣(經過轉換)
- iterm 分期期數

In [56]:
all_features.columns

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'loctm',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour',
       'loctm_min', 'loctm_sec', 'no', 'total_acc_sec', 'total_acc_min',
       'total_acc_hour', 'conam_is_0', 'cocam_u_cnt_by_acqic',
       'bacno_u_cnt_by_acqic', 'conam_level', 'loctm_value_counts',
       'loctm_hour_value_counts', 'csmcu_cnt_by_stocn', 'csmcu_cnt_by_scity',
       'csmcu_ucnt_by_bacno', 'bacno_value_counts', 'cano_value_counts',
       'mchno_value_counts', 'acqic_value_counts', 'mcc_value_counts',
       'ecfg_value_counts', 'insfg_value_counts', 'contp_value_counts',
       'etymd_value_counts', 'stocn_value_counts', 'scity_value_counts',
       'stscd_value_counts', 'ovrlt_value_counts', 'flbmk_value_counts',
       'hcefg_value_counts', 'csmcu_value_counts', 'flg_3dsmk_value_counts',
       'prime_time_normal', 'prime_time_fraud', 'sleep_

In [5]:
# "bacno", "cano", # 把交易日期當類別變數mean encoding on 交易金額 
cat_colnames = ["bacno", "cano", "mchno", "acqic", "mcc", "ecfg", "insfg", "contp", 'etymd', "iterm", 'loctm_hour', 'loctm_min',
       'loctm_sec', "stocn", "scity", "stscd", "ovrlt", "flbmk", "hcefg", "csmcu", "flg_3dsmk", "locdt"]
numeric_colnames = ["conam"]
# 不重要的特徵先隔離出來 --> 轉onehot的也先不加入groupby交易金額
unwanted_colnames = ['contp', 'iterm', 'ecfg', 'flbmk', 'insfg']
# unwanted_colnames = []

#### 超過2個level，但小於20個level的特徵轉one-hot

In [6]:
col_to_be_removed = []
for col in cat_colnames:
    if (all_features[col].unique().shape[0] > 2) and (all_features[col].unique().shape[0] < 20): 
        print(col)
        # print(all_features[col].unique())
        print(all_features[col].unique().shape[0])
        cat_one_hot = pd.get_dummies(all_features[col])
        cat_one_hot.columns = [col + "_" + str(cat) for cat in range(all_features[col].unique().shape[0])]
        all_features = pd.concat([all_features, cat_one_hot], axis=1)
        col_to_be_removed.append(col)
        unwanted_colnames.append(col)

print(all_features.shape)

for r_col in col_to_be_removed:
    del all_features[r_col]
    
print(all_features.shape)
print(unwanted_colnames)

contp
7
etymd
11
iterm
9
stscd
5
hcefg
10
(1938653, 100)
(1938653, 95)
['contp', 'iterm', 'ecfg', 'flbmk', 'insfg', 'contp', 'etymd', 'iterm', 'stscd', 'hcefg']


#### (特徵) 先把index浮出來當作key，groupby的時候會用到

In [5]:
# all_features = all_features.reset_index()
# all_features.head()

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,...,stscd_value_counts,ovrlt_value_counts,flbmk_value_counts,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000
0,0,6881,113261,38038,513.80,5,0,0,0,0,...,1915002,1913637,1932259,1799525,223907,1854121,1,1,0,0
1,1,0,134508,45725,465.62,5,0,0,2,0,...,1915002,1913637,1932259,67172,223907,1854121,1,0,0,0
2,2,6881,15408,188328,513.80,5,0,0,0,0,...,1915002,1913637,1932259,1799525,223907,1854121,1,1,0,0
3,3,6716,157159,29967,1016.11,5,62,0,5,0,...,1915002,1913637,1932259,1799525,1585893,1854121,1,1,0,0
4,4,5975,105985,81305,713.66,5,62,0,4,0,...,1915002,1913637,1932259,1799525,1585893,1854121,1,1,0,0


#### (特徵) 交易金額平均 (by feature mean encode)

In [59]:
for cat_col in cat_colnames:
    if cat_col not in unwanted_colnames:
        canam_cat_mean = all_features[[cat_col, "conam"]].groupby(cat_col, as_index=False).mean()
        canam_cat_mean = canam_cat_mean.rename(columns={"conam": "conam" + "_mean_by_" + cat_col})
        all_features = pd.merge(all_features, canam_cat_mean, left_on=cat_col, right_on=cat_col, how='left')
print(all_features.shape)
all_features.head(3)

(1938653, 109)


,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,conam_mean_by_mcc,conam_mean_by_loctm_hour,conam_mean_by_loctm_min,conam_mean_by_loctm_sec,conam_mean_by_stocn,conam_mean_by_scity,conam_mean_by_ovrlt,conam_mean_by_csmcu,conam_mean_by_flg_3dsmk,conam_mean_by_locdt
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,520.906246,644.916617,660.011903,650.872520,670.409075,488.192335,648.45153,484.287384,647.868952,744.039495
1,0,134508,45725,465.62,0,0,0,0,0,9,...,465.620000,619.386475,653.527407,649.432976,670.409075,676.955873,648.45153,484.287384,647.868952,649.387032
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,520.906246,695.431427,655.950615,653.588988,670.409075,488.192335,648.45153,484.287384,647.868952,644.352319


#### (特徵) 交易金額最大值 (by feature mean encode)

In [11]:
# for cat_col in cat_colnames:
#     if cat_col not in unwanted_colnames:
#         canam_cat_mean = all_features[[cat_col, "conam"]].groupby(cat_col, as_index=False).max()
#         canam_cat_mean = canam_cat_mean.rename(columns={"conam": "conam" + "_max_by_" + cat_col})
#         all_features = pd.merge(all_features, canam_cat_mean, left_on=cat_col, right_on=cat_col, how='left')
# print(all_features.shape)
# all_features.head(3)

(1938653, 124)


,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,conam_max_by_mcc,conam_max_by_locdt,conam_max_by_loctm_hour,conam_max_by_loctm_min,conam_max_by_loctm_sec,conam_max_by_stocn,conam_max_by_scity,conam_max_by_ovrlt,conam_max_by_csmcu,conam_max_by_flg_3dsmk
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,683.25,5075.54,4446.65,4438.05,5075.84,7208.77,3926.89,5866.36,2399.87,7208.77
1,1,0,134508,45725,465.62,0,0,0,0,0,...,465.62,3725.46,7195.84,4453.23,6728.97,7208.77,7208.77,5866.36,2399.87,7208.77
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,683.25,5975.93,5833.70,5278.28,4447.92,7208.77,3926.89,5866.36,2399.87,7208.77


#### 交易金額標準差

In [9]:
# for cat_col in cat_colnames:
#     # print(cat_col)
#     canam_cat_std = all_features[[cat_col, "conam"]].groupby(cat_col, as_index=False).std()
#     canam_cat_std = canam_cat_std.rename(columns={"conam": "conam" + "_std_by_" + cat_col})
#     all_features = pd.merge(all_features, canam_cat_std, left_on=cat_col, right_on=cat_col, how='left')
    
# print(all_features.shape)
# all_features.head(3)

#### 授權時間平均  (by feature mean encode)

In [6]:
# for cat_col in cat_colnames:
#     # print(cat_col)
#     if cat_col not in unwanted_colnames:
#         loctm_cat_mean = all_features[[cat_col, "loctm"]].groupby(cat_col, as_index=False).mean()
#         loctm_cat_mean = loctm_cat_mean.rename(columns={"loctm": "loctm" + "_mean_by_" + cat_col})
#         all_features = pd.merge(all_features, loctm_cat_mean, left_on=cat_col, right_on=cat_col, how='left')
    
# print(all_features.shape)
# all_features.head(3)

(1761621, 45)


,acqic,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,...,loctm_mean_by_mcc,loctm_mean_by_etymd,loctm_mean_by_locdt,loctm_mean_by_stocn,loctm_mean_by_scity,loctm_mean_by_stscd,loctm_mean_by_ovrlt,loctm_mean_by_hcefg,loctm_mean_by_csmcu,loctm_mean_by_flg_3dsmk
0,6881,513.80,5,0,0,0,0,0,5,0,...,148273.860703,143966.866639,138503.020000,147915.978536,143913.598253,146402.181976,146236.221644,147210.939985,135039.792814,146211.813969
1,0,465.62,5,0,0,2,0,0,0,0,...,103074.129415,130328.228719,150402.129153,147915.978536,143593.003056,146402.181976,146236.221644,103568.336250,135039.792814,146211.813969
2,6881,513.80,5,0,0,0,0,0,5,0,...,148273.860703,143966.866639,146146.379331,147915.978536,143913.598253,146402.181976,146236.221644,147210.939985,135039.792814,146211.813969


#### (特徵) 授權時間點counts平均

- 第一層: 授權時間點在整個資料集裡面出現的總次數
- 第二層: (以帳戶為例) 該帳戶每次消費的授權時間點在整個資料集裡面出現的總次數平均
- 直觀理解: 會跟授權時間點在整個資料集裡面出現次數相輔相成，等於該帳戶的1. 這個授權時間點常出現 2. 該帳戶授權時間點次數平均代表有沒有出現過一些低頻的授權時間點

In [5]:
imp_ftrs = ["bacno", "cano", "mchno", "conam", "acqic", "mcc", "scity", "loctm", "stscd", "csmcu"]
# loctm不會再跟自己做第二層了
for cat_col in cat_colnames:
    # print(cat_col)
    if cat_col not in unwanted_colnames and cat_col not in ['loctm_hour', 'loctm_min', 'loctm_sec'] \
    and cat_col in imp_ftrs:
        loctm_cnt_mean = all_features[[cat_col, "loctm_value_counts"]].groupby(cat_col, as_index=False).mean()
        loctm_cnt_mean = loctm_cnt_mean.rename(columns={"loctm_value_counts": "loctm_cnt" + "_mean_by_" + cat_col})
        all_features = pd.merge(all_features, loctm_cnt_mean, left_on=cat_col, right_on=cat_col, how='left')

print(all_features.shape)
all_features.head(3)

KeyboardInterrupt: 

#### 交易日期counts平均  (by feature mean encode)

In [8]:
# for cat_col in cat_colnames:
#     # print(cat_col)
#     if cat_col not in unwanted_colnames and cat_col != "locdt":
#         locdt_cnt_mean = all_features[[cat_col, "locdt_value_counts"]].groupby(cat_col, as_index=False).mean()
#         locdt_cnt_mean = locdt_cnt_mean.rename(columns={"locdt_value_counts": "locdt_cnt" + "_mean_by_" + cat_col})
#         all_features = pd.merge(all_features, locdt_cnt_mean, left_on=cat_col, right_on=cat_col, how='left')
    
# print(all_features.shape)
# all_features.head(3)

(1761621, 56)


,acqic,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,...,locdt_cnt_mean_by_acqic,locdt_cnt_mean_by_mcc,locdt_cnt_mean_by_etymd,locdt_cnt_mean_by_stocn,locdt_cnt_mean_by_scity,locdt_cnt_mean_by_stscd,locdt_cnt_mean_by_ovrlt,locdt_cnt_mean_by_hcefg,locdt_cnt_mean_by_csmcu,locdt_cnt_mean_by_flg_3dsmk
0,6881,513.80,5,0,0,0,0,0,5,0,...,14763.696375,14762.640789,14752.462178,14809.672615,14750.541979,14793.97767,14793.687306,14796.145570,14750.107908,14800.855787
1,0,465.62,5,0,0,2,0,0,0,0,...,14736.475697,14740.079289,14737.603884,14809.672615,14803.796875,14793.97767,14793.687306,14753.840474,14750.107908,14800.855787
2,6881,513.80,5,0,0,0,0,0,5,0,...,14763.696375,14762.640789,14752.462178,14809.672615,14750.541979,14793.97767,14793.687306,14796.145570,14750.107908,14800.855787


#### 授權時間標準差

In [8]:
# for cat_col in cat_colnames:
#     # print(cat_col)
#     loctm_cat_std = all_features[[cat_col, "loctm"]].groupby(cat_col, as_index=False).std()
#     loctm_cat_std = loctm_cat_std.rename(columns={"loctm": "loctm" + "_std_by_" + cat_col})
#     all_features = pd.merge(all_features, loctm_cat_std, left_on=cat_col, right_on=cat_col, how='left')

# print(all_features.shape)
# all_features.head(3)

#### (特徵) 帳號用不同卡交易總次數(by帳號看交易卡號出現次數)

In [61]:
cano_cnt_by_bacon = pd.DataFrame(all_features[["bacno", "cano"]].groupby("bacno").count()).reset_index()
cano_cnt_by_bacon.columns = ["bacno", "cano_cnt_by_bacon"]

all_features = pd.merge(all_features, cano_cnt_by_bacon, left_on='bacno', right_on="bacno", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,conam_mean_by_flg_3dsmk,conam_mean_by_locdt,loctm_cnt_mean_by_bacno,loctm_cnt_mean_by_cano,loctm_cnt_mean_by_mchno,loctm_cnt_mean_by_acqic,loctm_cnt_mean_by_mcc,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,647.868952,744.039495,32.612500,32.612500,28.789384,31.159090,31.252936,31.138777,36.959916,80
1,0,134508,45725,465.62,0,0,0,0,0,9,...,647.868952,649.387032,35.956522,35.956522,51.921808,52.530081,54.275977,33.615969,36.959916,23
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,647.868952,644.352319,27.055556,27.055556,28.789384,31.159090,31.252936,31.138777,36.959916,18


#### (特徵) 帳號&卡號綁一起的交易總次數

In [6]:
bacon_cano_cnt = pd.DataFrame(all_features[["bacno", "cano"]].groupby(["bacno", "cano"]).size()).reset_index()
bacon_cano_cnt.columns = ["bacno", "cano", "bacon_cano_cnt"]

all_features = pd.merge(all_features, bacon_cano_cnt, left_on=["bacno", "cano"], right_on=["bacno", "cano"], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,flbmk_value_counts,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1932259,1799525,223907,1854121,1,1,0,0,32.612500,80
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1932259,67172,223907,1854121,1,0,0,0,35.956522,23
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1932259,1799525,223907,1854121,1,1,0,0,27.055556,18


#### 帳戶該筆刷卡前的3次內累積金額 (後來綁帳戶&卡號)

In [69]:
# # locdt排序 then 算groupby bacno 前3筆
# bacno_3_times_conam_mean = all_features[["locdt", "bacno", "conam"]].sort_values("locdt").\
#                     groupby(["bacno"])["conam"].apply(lambda x: x[-4:-1].sum())
# bacno_3_times_conam_mean = pd.DataFrame(bacno_3_times_conam_mean).reset_index()
# bacno_3_times_conam_mean.columns = ["bacno", "bacno_3_times_conam_mean"]

# all_features = pd.merge(all_features, bacno_3_times_conam_mean, left_on='bacno', right_on="bacno", how='left')
# all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,conam_mean_by_flg_3dsmk,loctm_cnt_mean_by_bacno,loctm_cnt_mean_by_cano,loctm_cnt_mean_by_mchno,loctm_cnt_mean_by_acqic,loctm_cnt_mean_by_mcc,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacno_3_times_conam_mean
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,647.868952,32.612500,32.612500,28.789384,31.159090,31.252936,31.138777,36.959916,80,0.00
1,0,134508,45725,465.62,0,0,0,0,0,9,...,647.868952,35.956522,35.956522,51.921808,52.530081,54.275977,33.615969,36.959916,23,1396.86
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,647.868952,27.055556,27.055556,28.789384,31.159090,31.252936,31.138777,36.959916,18,1755.99


#### 帳戶該筆刷卡前的7次內累積金額 (後來綁帳戶&卡號)

In [64]:
# # locdt排序 then 算groupby bacno 前7筆
# bacno_7_times_conam_mean = all_features[["locdt", "bacno", "conam"]].sort_values("locdt").\
#                     groupby(["bacno"])["conam"].apply(lambda x: x[-8:-1].sum())
# bacno_7_times_conam_mean = pd.DataFrame(bacno_7_times_conam_mean).reset_index()
# bacno_7_times_conam_mean.columns = ["bacno", "bacno_7_times_conam_mean"]

# all_features = pd.merge(all_features, bacno_7_times_conam_mean, left_on='bacno', right_on="bacno", how='left')
# all_features.head(10)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,conam_mean_by_flg_3dsmk,loctm_cnt_mean_by_bacno,loctm_cnt_mean_by_cano,loctm_cnt_mean_by_mchno,loctm_cnt_mean_by_acqic,loctm_cnt_mean_by_mcc,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,647.676943,32.687500,32.687500,28.855059,31.222241,31.316191,31.202420,37.367194,80,80
1,1,0,134508,45725,465.62,0,0,0,0,0,...,647.676943,36.217391,36.217391,52.765172,53.384337,55.133381,33.781803,37.367194,23,23
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,647.676943,27.166667,27.166667,28.855059,31.222241,31.316191,31.202420,37.367194,18,18
3,3,6716,157159,29967,1016.11,62,0,0,0,0,...,647.676943,35.662162,37.421053,35.516385,33.152770,33.118993,34.414564,32.092869,74,57
4,4,5975,105985,81305,713.66,62,0,0,0,0,...,647.676943,34.615385,35.142857,31.500000,32.290119,31.442288,33.781803,32.092869,78,63
5,5,0,78377,2295,465.62,0,0,0,0,0,...,647.676943,38.178571,38.178571,52.765172,53.384337,55.133381,33.781803,37.367194,28,28
6,6,6411,94435,49219,1806.49,62,0,0,0,0,...,647.676943,29.666667,29.666667,52.765172,31.700787,30.553038,31.700787,32.092869,3,3
7,7,6769,112032,177989,526.88,62,0,0,0,0,...,647.676943,38.714286,38.714286,24.124153,32.767217,26.115805,33.781803,32.092869,84,84
8,8,6092,92294,85535,201.39,62,0,0,0,0,...,647.676943,29.000000,29.000000,28.905172,32.660938,30.966311,29.087248,32.092869,6,6
9,9,0,16279,110755,465.62,0,0,0,0,0,...,647.676943,49.833333,57.000000,52.765172,53.384337,55.133381,33.781803,37.367194,6,4


#### (特徵) 該帳戶該卡號該筆刷卡前所有金額累加

In [15]:
# bacno_cano_conam_cumsum = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x.cumsum())

# bacno_cano_conam_cumsum = pd.DataFrame(bacno_cano_conam_cumsum).reset_index()
# bacno_cano_conam_cumsum.columns = ["index", "bacno_cano_conam_cumsum"]
# all_features = pd.merge(all_features, bacno_cano_conam_cumsum, left_on="index", 
#                         right_on="index", how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,loctm_cnt_mean_by_bacno,loctm_cnt_mean_by_cano,loctm_cnt_mean_by_mchno,loctm_cnt_mean_by_acqic,loctm_cnt_mean_by_mcc,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,32.612500,32.612500,28.789384,31.159090,31.252936,31.138777,36.959916,80,80,12427.89
1,1,0,134508,45725,465.62,0,0,0,0,0,...,35.956522,35.956522,51.921808,52.530081,54.275977,33.615969,36.959916,23,23,2350.85
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,27.055556,27.055556,28.789384,31.159090,31.252936,31.138777,36.959916,18,18,513.80


#### (特徵) 該帳戶該卡號該筆刷卡前所有金額平均

In [16]:
bacno_cano_conam_cumsum_mean = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: cumsum_mean(x))

bacno_cano_conam_cumsum_mean = pd.DataFrame(bacno_cano_conam_cumsum_mean).reset_index()
bacno_cano_conam_cumsum_mean.columns = ["no", "bacno_cano_conam_cumsum_mean"]

all_features = pd.merge(all_features, bacno_cano_conam_cumsum_mean, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_7_times_conam_max_by_no,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_max_by_no,bacno_cano_conam_cumsum_mean
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,223907,1854121,1,1,0,0,2385.75,1458.33,2385.75,561.301096
1,0,134508,45725,465.62,5,0,0,2,0,0,...,223907,1854121,1,0,0,0,1881.93,812.26,1881.93,355.962083
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,223907,1854121,1,1,0,0,776.57,683.25,1304.96,552.063333


#### TODO: 包含自己by "天" 的金額加總、平均、最大值

#### (特徵) 該帳戶該卡號該筆刷卡的包含自己3次金額加總
https://blog.csdn.net/wj1066/article/details/78853717 

In [7]:
bacno_cano_conam_rolling_3_window_sum = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(3, min_periods=1).sum())

bacno_cano_conam_rolling_3_window_sum = pd.DataFrame(bacno_cano_conam_rolling_3_window_sum).reset_index()
bacno_cano_conam_rolling_3_window_sum.columns = ["no", "bacno_cano_conam_rolling_3_window_sum"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_3_window_sum, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1799525,223907,1854121,1,1,0,0,32.612500,80,1725.89
1,0,134508,45725,465.62,5,0,0,2,0,0,...,67172,223907,1854121,1,0,0,0,35.956522,23,703.98
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1799525,223907,1854121,1,1,0,0,27.055556,18,1650.12


#### (特徵) 該帳戶該卡號該筆刷卡的包含自己3次金額平均

In [8]:
bacno_cano_conam_rolling_3_window_mean = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(3, min_periods=1).mean())

bacno_cano_conam_rolling_3_window_mean = pd.DataFrame(bacno_cano_conam_rolling_3_window_mean).reset_index()
bacno_cano_conam_rolling_3_window_mean.columns = ["no", "bacno_cano_conam_rolling_3_window_mean"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_3_window_mean, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,223907,1854121,1,1,0,0,32.612500,80,1725.89,575.296667
1,0,134508,45725,465.62,5,0,0,2,0,0,...,223907,1854121,1,0,0,0,35.956522,23,703.98,234.660000
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,223907,1854121,1,1,0,0,27.055556,18,1650.12,550.040000


#### (特徵) 該帳戶該卡號該筆刷卡的最後幾天挑3筆金額加總 -> 盜刷後停卡

In [19]:
# locdt排序 then 算groupby bacno, cano 前3筆
# bacno_cano_3_times_conam_sum = all_features[["locdt", "bacno", "cano", "conam"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-4:-1].sum())

# bacno_cano_3_times_conam_sum = pd.DataFrame(bacno_cano_3_times_conam_sum).reset_index()
# bacno_cano_3_times_conam_sum.columns = ["bacno", "cano", "bacno_cano_3_times_conam_sum"]

# all_features = pd.merge(all_features, bacno_cano_3_times_conam_sum, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,loctm_cnt_mean_by_mcc,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_3_times_conam_sum
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,31.252936,31.138777,36.959916,80,80,12427.89,564.904091,1541.40,513.80,0.00
1,1,0,134508,45725,465.62,0,0,0,0,0,...,54.275977,33.615969,36.959916,23,23,2350.85,587.712500,1765.29,588.43,1396.86
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,31.252936,31.138777,36.959916,18,18,513.80,513.800000,513.80,513.80,1755.99


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後3筆金額加總

In [20]:
# locdt排序 then 算groupby bacno, cano 前3筆
# bacno_cano_3_times_conam_sum_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-4:-1].sum())

# bacno_cano_3_times_conam_sum_by_no = pd.DataFrame(bacno_cano_3_times_conam_sum_by_no).reset_index()
# bacno_cano_3_times_conam_sum_by_no.columns = ["bacno", "cano", "bacno_cano_3_times_conam_sum_by_no"]

# all_features = pd.merge(all_features, bacno_cano_3_times_conam_sum_by_no, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_3_times_conam_sum,bacno_cano_3_times_conam_sum_by_no
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,31.138777,36.959916,80,80,12427.89,564.904091,1541.40,513.80,0.00,2385.75
1,1,0,134508,45725,465.62,0,0,0,0,0,...,33.615969,36.959916,23,23,2350.85,587.712500,1765.29,588.43,1396.86,1396.86
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,31.138777,36.959916,18,18,513.80,513.800000,513.80,513.80,1755.99,1755.99


#### (特徵) 該帳戶該卡號該筆刷卡的包含自己7次金額加總

In [9]:
bacno_cano_conam_rolling_7_window_sum = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(7, min_periods=1).sum())

bacno_cano_conam_rolling_7_window_sum = pd.DataFrame(bacno_cano_conam_rolling_7_window_sum).reset_index()
bacno_cano_conam_rolling_7_window_sum.columns = ["no", "bacno_cano_conam_rolling_7_window_sum"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_7_window_sum, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1854121,1,1,0,0,32.612500,80,1725.89,575.296667,3673.51
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1854121,1,0,0,0,35.956522,23,703.98,234.660000,2482.74
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1854121,1,1,0,0,27.055556,18,1650.12,550.040000,3892.49


#### (特徵) 該帳戶該卡號該筆刷卡的包含自己7次金額平均

In [10]:
bacno_cano_conam_rolling_7_window_mean = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(7, min_periods=1).mean())

bacno_cano_conam_rolling_7_window_mean = pd.DataFrame(bacno_cano_conam_rolling_7_window_mean).reset_index()
bacno_cano_conam_rolling_7_window_mean.columns = ["no", "bacno_cano_conam_rolling_7_window_mean"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_7_window_mean, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1,1,0,0,32.612500,80,1725.89,575.296667,3673.51,524.787143
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1,0,0,0,35.956522,23,703.98,234.660000,2482.74,354.677143
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1,1,0,0,27.055556,18,1650.12,550.040000,3892.49,556.070000


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後7筆金額加總

In [12]:
bacno_cano_7_times_conam_sum_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-7:].sum())

bacno_cano_7_times_conam_sum_by_no = pd.DataFrame(bacno_cano_7_times_conam_sum_by_no).reset_index()
bacno_cano_7_times_conam_sum_by_no.columns = ["bacno", "cano", "bacno_cano_7_times_conam_sum_by_no"]

all_features = pd.merge(all_features, bacno_cano_7_times_conam_sum_by_no, left_on=['bacno', 'cano'], 
                        right_on=['bacno', 'cano'], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_7_times_conam_sum_by_no
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1,0,0,32.612500,80,1725.89,575.296667,3673.51,524.787143,3258.80
1,0,134508,45725,465.62,5,0,0,2,0,0,...,0,0,0,35.956522,23,703.98,234.660000,2482.74,354.677143,4723.83
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1,0,0,27.055556,18,1650.12,550.040000,3892.49,556.070000,4254.47


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後7筆金額平均

In [13]:
bacno_cano_7_times_conam_mean_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-7:].mean())

bacno_cano_7_times_conam_mean_by_no = pd.DataFrame(bacno_cano_7_times_conam_mean_by_no).reset_index()
bacno_cano_7_times_conam_mean_by_no.columns = ["bacno", "cano", "bacno_cano_7_times_conam_mean_by_no"]

all_features = pd.merge(all_features, bacno_cano_7_times_conam_mean_by_no, left_on=['bacno', 'cano'], 
                        right_on=['bacno', 'cano'], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,sleep_time,sleep_time_conam_lower_3000,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_7_times_conam_sum_by_no,bacno_cano_7_times_conam_mean_by_no
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,0,0,32.612500,80,1725.89,575.296667,3673.51,524.787143,3258.80,465.542857
1,0,134508,45725,465.62,5,0,0,2,0,0,...,0,0,35.956522,23,703.98,234.660000,2482.74,354.677143,4723.83,674.832857
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,0,0,27.055556,18,1650.12,550.040000,3892.49,556.070000,4254.47,607.781429


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後7筆金額最大值

In [13]:
bacno_cano_7_times_conam_max_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-7:].max())

bacno_cano_7_times_conam_max_by_no = pd.DataFrame(bacno_cano_7_times_conam_max_by_no).reset_index()
bacno_cano_7_times_conam_max_by_no.columns = ["bacno", "cano", "bacno_cano_7_times_conam_max_by_no"]

all_features = pd.merge(all_features, bacno_cano_7_times_conam_max_by_no, left_on=['bacno', 'cano'], 
                        right_on=['bacno', 'cano'], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,ovrlt_value_counts,flbmk_value_counts,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_7_times_conam_max_by_no
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1913637,1932259,1799525,223907,1854121,1,1,0,0,2385.75
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1913637,1932259,67172,223907,1854121,1,0,0,0,1881.93
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1913637,1932259,1799525,223907,1854121,1,1,0,0,776.57


#### (特徵) 該帳戶該卡號最後幾天挑7筆金額加總

In [26]:
# locdt排序 then 算groupby bacno, cano 前7筆
# bacno_cano_7_times_conam_sum = all_features[["locdt", "bacno", "cano", "conam"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-7:].sum())

# bacno_cano_7_times_conam_sum = pd.DataFrame(bacno_cano_7_times_conam_sum).reset_index()
# bacno_cano_7_times_conam_sum.columns = ["bacno", "cano", "bacno_cano_7_times_conam_sum"]

# all_features = pd.merge(all_features, bacno_cano_7_times_conam_sum, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_3_times_conam_sum,bacno_cano_3_times_conam_sum_by_no,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_7_times_conam_sum_by_no,bacno_cano_7_times_conam_mean_by_no,bacno_cano_7_times_conam_max_by_no,bacno_cano_7_times_conam_sum
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,1541.40,513.80,0.00,2385.75,3875.21,553.601429,3258.80,465.542857,2385.75,3258.80
1,1,0,134508,45725,465.62,0,0,0,0,0,...,1765.29,588.43,1396.86,1396.86,2350.85,587.712500,4723.83,674.832857,1881.93,4723.83
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,513.80,513.80,1755.99,1755.99,513.80,513.800000,4254.47,607.781429,776.57,4254.47


#### (特徵) 該帳戶該卡號最後幾天挑7筆金額平均

In [27]:
# locdt排序 then 算groupby bacno, cano 前7筆
# bacno_cano_7_times_conam_mean = all_features[["locdt", "bacno", "cano", "conam"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-7:].mean())

# bacno_cano_7_times_conam_mean = pd.DataFrame(bacno_cano_7_times_conam_mean).reset_index()
# bacno_cano_7_times_conam_mean.columns = ["bacno", "cano", "bacno_cano_7_times_conam_mean"]

# all_features = pd.merge(all_features, bacno_cano_7_times_conam_mean, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_conam_rolling_3_window_mean,bacno_cano_3_times_conam_sum,bacno_cano_3_times_conam_sum_by_no,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_7_times_conam_sum_by_no,bacno_cano_7_times_conam_mean_by_no,bacno_cano_7_times_conam_max_by_no,bacno_cano_7_times_conam_sum,bacno_cano_7_times_conam_mean
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,513.80,0.00,2385.75,3875.21,553.601429,3258.80,465.542857,2385.75,3258.80,465.542857
1,1,0,134508,45725,465.62,0,0,0,0,0,...,588.43,1396.86,1396.86,2350.85,587.712500,4723.83,674.832857,1881.93,4723.83,674.832857
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,513.80,1755.99,1755.99,513.80,513.800000,4254.47,607.781429,776.57,4254.47,607.781429


 #### (特徵) 該帳戶該卡號最後幾天挑7筆內最高金額

In [28]:
# locdt排序 then 算groupby bacno, cano 前7筆
# bacno_cano_7_times_conam_max = all_features[["locdt", "bacno", "cano", "conam"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-7:].max())

# bacno_cano_7_times_conam_max = pd.DataFrame(bacno_cano_7_times_conam_max).reset_index()
# bacno_cano_7_times_conam_max.columns = ["bacno", "cano", "bacno_cano_7_times_conam_max"]

# all_features = pd.merge(all_features, bacno_cano_7_times_conam_max, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_3_times_conam_sum,bacno_cano_3_times_conam_sum_by_no,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_7_times_conam_sum_by_no,bacno_cano_7_times_conam_mean_by_no,bacno_cano_7_times_conam_max_by_no,bacno_cano_7_times_conam_sum,bacno_cano_7_times_conam_mean,bacno_cano_7_times_conam_max
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,0.00,2385.75,3875.21,553.601429,3258.80,465.542857,2385.75,3258.80,465.542857,2385.75
1,1,0,134508,45725,465.62,0,0,0,0,0,...,1396.86,1396.86,2350.85,587.712500,4723.83,674.832857,1881.93,4723.83,674.832857,1881.93
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,1755.99,1755.99,513.80,513.800000,4254.47,607.781429,776.57,4254.47,607.781429,776.57


 #### (特徵) 該帳戶該卡號該筆刷卡的包含自己14次金額加總

In [15]:
bacno_cano_conam_rolling_14_window_sum = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(14, min_periods=1).sum())

bacno_cano_conam_rolling_14_window_sum = pd.DataFrame(bacno_cano_conam_rolling_14_window_sum).reset_index()
bacno_cano_conam_rolling_14_window_sum.columns = ["no", "bacno_cano_conam_rolling_14_window_sum"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_14_window_sum, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,loctm_cnt_mean_by_bacno,bacon_cano_cnt,bacno_cano_conam_rolling_3_window_sum,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_7_times_conam_sum_by_no,bacno_cano_7_times_conam_mean_by_no,bacno_cano_7_times_conam_max_by_no,bacno_cano_conam_rolling_14_window_sum
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,32.612500,80,1725.89,575.296667,3673.51,524.787143,3258.80,465.542857,2385.75,8422.11
1,0,134508,45725,465.62,5,0,0,2,0,0,...,35.956522,23,703.98,234.660000,2482.74,354.677143,4723.83,674.832857,1881.93,5365.48
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,27.055556,18,1650.12,550.040000,3892.49,556.070000,4254.47,607.781429,776.57,7959.98


 #### (特徵) 該帳戶該卡號該筆刷卡的包含自己14次金額平均

In [67]:
bacno_cano_conam_rolling_14_window_mean = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(14, min_periods=1).mean())

bacno_cano_conam_rolling_14_window_mean = pd.DataFrame(bacno_cano_conam_rolling_14_window_mean).reset_index()
bacno_cano_conam_rolling_14_window_mean.columns = ["no", "bacno_cano_conam_rolling_14_window_mean"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_14_window_mean, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,loctm_cnt_mean_by_mcc,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,31.252936,31.138777,36.959916,80,80,561.301096,575.296667,3673.51,524.787143,601.579286
1,0,134508,45725,465.62,0,0,0,0,0,9,...,54.275977,33.615969,36.959916,23,23,355.962083,234.660000,2482.74,354.677143,383.248571
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,31.252936,31.138777,36.959916,18,18,552.063333,550.040000,3892.49,556.070000,568.570000


In [ ]:
del all_features["bacno_cano_conam_rolling_14_window_mean"]

 #### (特徵) 該帳戶該卡號該筆刷卡的包含自己14次金額最大值

In [14]:
bacno_cano_conam_rolling_14_window_max = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.rolling(14, min_periods=1).max())

bacno_cano_conam_rolling_14_window_max = pd.DataFrame(bacno_cano_conam_rolling_14_window_max).reset_index()
bacno_cano_conam_rolling_14_window_max.columns = ["no", "bacno_cano_conam_rolling_14_window_max"]

all_features = pd.merge(all_features, bacno_cano_conam_rolling_14_window_max, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,flbmk_value_counts,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_7_times_conam_max_by_no,bacno_cano_conam_rolling_14_window_max
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1932259,1799525,223907,1854121,1,1,0,0,2385.75,1458.33
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1932259,67172,223907,1854121,1,0,0,0,1881.93,812.26
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1932259,1799525,223907,1854121,1,1,0,0,776.57,683.25


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後14筆金額加總

In [32]:
# bacno_cano_14_times_conam_sum_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-14:].sum())

# bacno_cano_14_times_conam_sum_by_no = pd.DataFrame(bacno_cano_14_times_conam_sum_by_no).reset_index()
# bacno_cano_14_times_conam_sum_by_no.columns = ["bacno", "cano", "bacno_cano_14_times_conam_sum_by_no"]

# all_features = pd.merge(all_features, bacno_cano_14_times_conam_sum_by_no, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_7_times_conam_sum_by_no,bacno_cano_7_times_conam_mean_by_no,bacno_cano_7_times_conam_max_by_no,bacno_cano_7_times_conam_sum,bacno_cano_7_times_conam_mean,bacno_cano_7_times_conam_max,bacno_cano_conam_rolling_14_window_sum,bacno_cano_conam_rolling_14_window_mean,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_sum_by_no
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,3258.80,465.542857,2385.75,3258.80,465.542857,2385.75,8305.91,593.279286,1038.68,6431.80
1,1,0,134508,45725,465.62,0,0,0,0,0,...,4723.83,674.832857,1881.93,4723.83,674.832857,1881.93,2350.85,587.712500,664.07,8611.46
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,4254.47,607.781429,776.57,4254.47,607.781429,776.57,513.80,513.800000,513.80,9413.80


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後14筆金額平均

In [33]:
# bacno_cano_14_times_conam_mean_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-14:].mean())

# bacno_cano_14_times_conam_mean_by_no = pd.DataFrame(bacno_cano_14_times_conam_mean_by_no).reset_index()
# bacno_cano_14_times_conam_mean_by_no.columns = ["bacno", "cano", "bacno_cano_14_times_conam_mean_by_no"]

# all_features = pd.merge(all_features, bacno_cano_14_times_conam_mean_by_no, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_7_times_conam_mean_by_no,bacno_cano_7_times_conam_max_by_no,bacno_cano_7_times_conam_sum,bacno_cano_7_times_conam_mean,bacno_cano_7_times_conam_max,bacno_cano_conam_rolling_14_window_sum,bacno_cano_conam_rolling_14_window_mean,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_sum_by_no,bacno_cano_14_times_conam_mean_by_no
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,465.542857,2385.75,3258.80,465.542857,2385.75,8305.91,593.279286,1038.68,6431.80,459.414286
1,1,0,134508,45725,465.62,0,0,0,0,0,...,674.832857,1881.93,4723.83,674.832857,1881.93,2350.85,587.712500,664.07,8611.46,615.104286
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,607.781429,776.57,4254.47,607.781429,776.57,513.80,513.800000,513.80,9413.80,672.414286


#### (特徵) 該帳戶該卡號該筆刷卡的挑最後14筆金額最大值

In [15]:
bacno_cano_14_times_conam_max_by_no = all_features[["no", "bacno", "cano", "conam"]].sort_values("no").\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-14:].max())

bacno_cano_14_times_conam_max_by_no = pd.DataFrame(bacno_cano_14_times_conam_max_by_no).reset_index()
bacno_cano_14_times_conam_max_by_no.columns = ["bacno", "cano", "bacno_cano_14_times_conam_max_by_no"]

all_features = pd.merge(all_features, bacno_cano_14_times_conam_max_by_no, left_on=["bacno", "cano"], 
                        right_on=["bacno", "cano"], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_7_times_conam_max_by_no,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_max_by_no
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1799525,223907,1854121,1,1,0,0,2385.75,1458.33,2385.75
1,0,134508,45725,465.62,5,0,0,2,0,0,...,67172,223907,1854121,1,0,0,0,1881.93,812.26,1881.93
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1799525,223907,1854121,1,1,0,0,776.57,683.25,1304.96


#### (特徵) 該帳戶該卡號最後幾天挑14筆金額平均

In [35]:
# # locdt排序 then 算groupby bacno, cano 前14筆
# bacno_cano_14_times_conam_mean = all_features[["locdt", "bacno", "cano", "conam"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-14:].mean())

# bacno_cano_14_times_conam_mean = pd.DataFrame(bacno_cano_14_times_conam_mean).reset_index()
# bacno_cano_14_times_conam_mean.columns = ["bacno", "cano", "bacno_cano_14_times_conam_mean"]

# all_features = pd.merge(all_features, bacno_cano_14_times_conam_mean, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_7_times_conam_sum,bacno_cano_7_times_conam_mean,bacno_cano_7_times_conam_max,bacno_cano_conam_rolling_14_window_sum,bacno_cano_conam_rolling_14_window_mean,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_sum_by_no,bacno_cano_14_times_conam_mean_by_no,bacno_cano_14_times_conam_max_by_no,bacno_cano_14_times_conam_mean
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,3258.80,465.542857,2385.75,8305.91,593.279286,1038.68,6431.80,459.414286,2385.75,458.882143
1,1,0,134508,45725,465.62,0,0,0,0,0,...,4723.83,674.832857,1881.93,2350.85,587.712500,664.07,8611.46,615.104286,1881.93,615.104286
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,4254.47,607.781429,776.57,513.80,513.800000,513.80,9413.80,672.414286,1304.96,672.414286


#### (特徵) 帳號卡號最後幾天挑14筆交易的交易時間發生在盜刷熱門時段的次數

In [ ]:
# bacno_cano_14_times_prime_time_fraud_cnt = all_features[["locdt", "bacno", "cano", "prime_time_fraud"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["prime_time_fraud"].apply(lambda x: x[-14:].sum())

# bacno_cano_14_times_prime_time_fraud_cnt = pd.DataFrame(bacno_cano_14_times_prime_time_fraud_cnt).reset_index()
# bacno_cano_14_times_prime_time_fraud_cnt.columns = ["bacno", "cano", "bacno_cano_14_times_prime_time_fraud_cnt"]

# all_features = pd.merge(all_features, bacno_cano_14_times_prime_time_fraud_cnt, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

#### (特徵) 帳號卡號最近幾筆14筆交易內的交易時間發生在盜刷熱門時段的次數

In [68]:
bacno_cano_14_times_prime_time_fraud_cnt = all_features[["no", "bacno", "cano", "prime_time_fraud"]].sort_values("no").\
                    groupby(["bacno", "cano"])["prime_time_fraud"].apply(lambda x: x.rolling(14, min_periods=1).sum())

bacno_cano_14_times_prime_time_fraud_cnt = pd.DataFrame(bacno_cano_14_times_prime_time_fraud_cnt).reset_index()
bacno_cano_14_times_prime_time_fraud_cnt.columns = ["no", "bacno_cano_14_rolling_times_prime_time_fraud_cnt"]

all_features = pd.merge(all_features, bacno_cano_14_times_prime_time_fraud_cnt, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,loctm_cnt_mean_by_scity,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,31.138777,36.959916,80,80,561.301096,575.296667,3673.51,524.787143,601.579286,12.0
1,0,134508,45725,465.62,0,0,0,0,0,9,...,33.615969,36.959916,23,23,355.962083,234.660000,2482.74,354.677143,383.248571,14.0
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,31.138777,36.959916,18,18,552.063333,550.040000,3892.49,556.070000,568.570000,10.0


#### (特徵) 帳號卡號最後幾天挑14筆交易的交易時間發生在睡覺時間的次數

In [56]:
# bacno_cano_14_times_sleep_time_cnt = all_features[["locdt", "bacno", "cano", "sleep_time"]].sort_values("locdt").\
#                     groupby(["bacno", "cano"])["sleep_time"].apply(lambda x: x[-14:].sum())

# bacno_cano_14_times_sleep_time_cnt = pd.DataFrame(bacno_cano_14_times_sleep_time_cnt).reset_index()
# bacno_cano_14_times_sleep_time_cnt.columns = ["bacno", "cano", "bacno_cano_14_times_sleep_time_cnt"]

# all_features = pd.merge(all_features, bacno_cano_14_times_sleep_time_cnt, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_7_times_conam_max,bacno_cano_conam_rolling_14_window_sum,bacno_cano_conam_rolling_14_window_mean,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_sum_by_no,bacno_cano_14_times_conam_mean_by_no,bacno_cano_14_times_conam_max_by_no,bacno_cano_14_times_conam_mean,bacno_cano_7_times_prime_time_fraud_cnt,bacno_cano_7_times_sleep_time_cnt
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,2385.75,8305.91,593.279286,1038.68,6431.80,459.414286,2385.75,496.036429,11,1
1,1,0,134508,45725,465.62,0,0,0,0,0,...,1881.93,2350.85,587.712500,664.07,8611.46,615.104286,1881.93,615.104286,7,1
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,776.57,513.80,513.800000,513.80,9413.80,672.414286,1304.96,672.414286,13,0


#### (特徵) 帳號卡號最近14筆交易的交易時間發生在睡覺時間的次數

In [69]:
bacno_cano_14_times_sleep_time_cnt = all_features[["no", "bacno", "cano", "sleep_time"]].sort_values("no").\
                    groupby(["bacno", "cano"])["sleep_time"].apply(lambda x: x.rolling(14, min_periods=1).sum())

bacno_cano_14_times_sleep_time_cnt = pd.DataFrame(bacno_cano_14_times_sleep_time_cnt).reset_index()
bacno_cano_14_times_sleep_time_cnt.columns = ["no", "bacno_cano_14_times_sleep_time_cnt"]

all_features = pd.merge(all_features, bacno_cano_14_times_sleep_time_cnt, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,loctm_cnt_mean_by_csmcu,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,36.959916,80,80,561.301096,575.296667,3673.51,524.787143,601.579286,12.0,1.0
1,0,134508,45725,465.62,0,0,0,0,0,9,...,36.959916,23,23,355.962083,234.660000,2482.74,354.677143,383.248571,14.0,0.0
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,36.959916,18,18,552.063333,550.040000,3892.49,556.070000,568.570000,10.0,0.0


#### (特徵) 帳號卡號最後幾天挑14筆交易的交易時間發生在睡覺時間且金額小於3000的次數

In [70]:
bacno_cano_7_sleep_time_conam_lower_3000_cnt = all_features[["locdt", "bacno", "cano", "sleep_time_conam_lower_3000"]].sort_values("locdt").\
                    groupby(["bacno", "cano"])["sleep_time_conam_lower_3000"].apply(lambda x: x[-14:].sum())

bacno_cano_7_sleep_time_conam_lower_3000_cnt = pd.DataFrame(bacno_cano_7_sleep_time_conam_lower_3000_cnt).reset_index()
bacno_cano_7_sleep_time_conam_lower_3000_cnt.columns = ["bacno", "cano", "bacno_cano_7_sleep_time_conam_lower_3000_cnt"]

all_features = pd.merge(all_features, bacno_cano_7_sleep_time_conam_lower_3000_cnt, left_on=['bacno', 'cano'], 
                        right_on=['bacno', 'cano'], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,cano_cnt_by_bacon,bacon_cano_cnt,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,80,80,561.301096,575.296667,3673.51,524.787143,601.579286,12.0,1.0,0
1,0,134508,45725,465.62,0,0,0,0,0,9,...,23,23,355.962083,234.660000,2482.74,354.677143,383.248571,14.0,0.0,1
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,18,18,552.063333,550.040000,3892.49,556.070000,568.570000,10.0,0.0,0


In [17]:
# del all_features["bacno_cano_7_sleep_time_conam_lower_3000_cnt"]

#### (特徵) 帳號卡號最近14筆交易的交易時間發生在睡覺時間且金額小於3000的次數

In [72]:
bacno_cano_14_sleep_time_conam_lower_3000_cnt = all_features[["no", "bacno", "cano", "sleep_time_conam_lower_3000"]].sort_values("no").\
                    groupby(["bacno", "cano"])["sleep_time_conam_lower_3000"].apply(lambda x: x.rolling(14, min_periods=1).sum())

bacno_cano_14_sleep_time_conam_lower_3000_cnt = pd.DataFrame(bacno_cano_14_sleep_time_conam_lower_3000_cnt).reset_index()
bacno_cano_14_sleep_time_conam_lower_3000_cnt.columns = ["no", "bacno_cano_14_sleep_time_conam_lower_3000_cnt"]

all_features = pd.merge(all_features, bacno_cano_14_sleep_time_conam_lower_3000_cnt, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,bacon_cano_cnt,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,80,561.301096,575.296667,3673.51,524.787143,601.579286,12.0,1.0,0,1.0
1,0,134508,45725,465.62,0,0,0,0,0,9,...,23,355.962083,234.660000,2482.74,354.677143,383.248571,14.0,0.0,1,0.0
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,18,552.063333,550.040000,3892.49,556.070000,568.570000,10.0,0.0,0,0.0


#### TODO: 帳戶該筆交易前所有交易消費金額diff是不是平均為正

#### (特徵) 帳戶所有交易消費金額diff是不是平均為正

In [40]:
# bacno_cano_no_all_conam_pos_trend = all_features[["no", "bacno", "cano", "conam"]].sort_values(["no"]).\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x.diff().fillna(0).mean())

# bacno_cano_no_all_conam_pos_trend = pd.DataFrame(bacno_cano_no_all_conam_pos_trend).reset_index()
# bacno_cano_no_all_conam_pos_trend.columns = ["bacno", "cano", "bacno_cano_no_all_conam_pos_trend"]

# all_features = pd.merge(all_features, bacno_cano_no_all_conam_pos_trend, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_sum_by_no,bacno_cano_14_times_conam_mean_by_no,bacno_cano_14_times_conam_max_by_no,bacno_cano_14_times_conam_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_all_conam_pos_trend
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,1038.68,6431.80,459.414286,2385.75,458.882143,11.0,0.0,0,0.0,-8.237000
1,1,0,134508,45725,465.62,0,0,0,0,0,...,664.07,8611.46,615.104286,1881.93,615.104286,2.0,1.0,1,1.0,-3.120000
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,513.80,9413.80,672.414286,1304.96,672.414286,1.0,0.0,0,0.0,10.227222


#### (特徵) 最後14次的交易金額的diff是不是平均為正

In [41]:
# bacno_cano_no_14_conam_pos_trend = all_features[["no", "bacno", "cano", "conam"]].sort_values(["no"]).\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-15:].diff().fillna(0).mean())

# bacno_cano_no_14_conam_pos_trend = pd.DataFrame(bacno_cano_no_14_conam_pos_trend).reset_index()
# bacno_cano_no_14_conam_pos_trend.columns = ["bacno", "cano", "bacno_cano_no_14_conam_pos_trend"]

# all_features = pd.merge(all_features, bacno_cano_no_14_conam_pos_trend, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_14_times_conam_sum_by_no,bacno_cano_14_times_conam_mean_by_no,bacno_cano_14_times_conam_max_by_no,bacno_cano_14_times_conam_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_all_conam_pos_trend,bacno_cano_no_14_conam_pos_trend
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,6431.80,459.414286,2385.75,458.882143,11.0,0.0,0,0.0,-8.237000,-7.243333
1,1,0,134508,45725,465.62,0,0,0,0,0,...,8611.46,615.104286,1881.93,615.104286,2.0,1.0,1,1.0,-3.120000,-45.576000
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,9413.80,672.414286,1304.96,672.414286,1.0,0.0,0,0.0,10.227222,-13.696667


#### (特徵) 最近7次的交易金額的diff是不是平均為正

In [73]:
bacno_cano_no_recent_7_conam_pos_trend = all_features[["no", "bacno", "cano", "conam"]].sort_values(["no"]).\
                    groupby(["bacno", "cano"])["conam"].apply(lambda x: x.diff().fillna(0).rolling(7, min_periods=1).mean())

bacno_cano_no_recent_7_conam_pos_trend = pd.DataFrame(bacno_cano_no_recent_7_conam_pos_trend).reset_index()
bacno_cano_no_recent_7_conam_pos_trend.columns = ["no", "bacno_cano_no_recent_7_conam_pos_trend"]

all_features = pd.merge(all_features, bacno_cano_no_recent_7_conam_pos_trend, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,bacno_cano_conam_cumsum_mean,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_recent_7_conam_pos_trend
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,561.301096,575.296667,3673.51,524.787143,601.579286,12.0,1.0,0,1.0,4.060244e-14
1,0,134508,45725,465.62,0,0,0,0,0,9,...,355.962083,234.660000,2482.74,354.677143,383.248571,14.0,0.0,1,0.0,-5.442000e+01
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,552.063333,550.040000,3892.49,556.070000,568.570000,10.0,0.0,0,0.0,-1.809000e+01


#### (特徵) 最後幾天14次的交易金額的diff是不是平均為正

In [60]:
# bacno_cano_locdt_tm_14_conam_pos_trend = all_features[["locdt", "loctm", "bacno", "cano", "conam"]].sort_values(["locdt", "loctm"]).\
#                     groupby(["bacno", "cano"])["conam"].apply(lambda x: x[-15:].diff().fillna(0).mean())

# bacno_cano_locdt_tm_14_conam_pos_trend = pd.DataFrame(bacno_cano_locdt_tm_14_conam_pos_trend).reset_index()
# bacno_cano_locdt_tm_14_conam_pos_trend.columns = ["bacno", "cano", "bacno_cano_locdt_tm_14_conam_pos_trend"]

# all_features = pd.merge(all_features, bacno_cano_locdt_tm_14_conam_pos_trend, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_14_times_conam_sum_by_no,bacno_cano_14_times_conam_mean_by_no,bacno_cano_14_times_conam_max_by_no,bacno_cano_14_times_conam_mean,bacno_cano_7_times_prime_time_fraud_cnt,bacno_cano_7_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_no_all_conam_pos_trend,bacno_cano_no_14_conam_pos_trend,bacno_cano_locdt_tm_14_conam_pos_trend
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,6431.80,459.414286,2385.75,496.036429,11,1,1,-8.237000,-7.243333,-7.243333
1,1,0,134508,45725,465.62,0,0,0,0,0,...,8611.46,615.104286,1881.93,615.104286,7,1,1,-3.120000,-45.576000,-45.576000
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,9413.80,672.414286,1304.96,672.414286,13,0,0,10.227222,-13.696667,-13.696667


#### TODO: 交易前數次的時間差平均
#### TODO: 該筆與同帳戶卡號交易前後各一筆的時間差取小

#### (特徵) 該筆與同帳戶卡號交易後一筆減前一筆的秒數

In [74]:
bacno_cano_sec_diff_latter = all_features[["no", "bacno", "cano", "total_acc_sec"]].sort_values(["no"]).\
                        groupby(["bacno", "cano"])["total_acc_sec"]\
                        .apply(lambda x: x.rolling(2).apply(lambda x: x[1] - x[0]).fillna(0))
bacno_cano_sec_diff_latter = pd.DataFrame(bacno_cano_sec_diff_latter).reset_index()
bacno_cano_sec_diff_latter.columns = ["no" ,"bacno_cano_sec_diff_latter"]

all_features = pd.merge(all_features, bacno_cano_sec_diff_latter, left_on="no", 
                        right_on="no", how='left')
all_features.head(3)

/home/rossleecooloh/.pyenv/versions/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until


,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,bacno_cano_conam_rolling_3_window_mean,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_recent_7_conam_pos_trend,bacno_cano_sec_diff_latter
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,575.296667,3673.51,524.787143,601.579286,12.0,1.0,0,1.0,4.060244e-14,9575.0
1,0,134508,45725,465.62,0,0,0,0,0,9,...,234.660000,2482.74,354.677143,383.248571,14.0,0.0,1,0.0,-5.442000e+01,4966.0
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,550.040000,3892.49,556.070000,568.570000,10.0,0.0,0,0.0,-1.809000e+01,76914.0


#### (特徵) 該筆與同帳戶卡號交易後一筆減前一筆的分鐘數 (跟秒基本上一樣@@)

In [ ]:
# bacno_cano_min_diff_latter = all_features[["no", "bacno", "cano", "total_acc_min"]].sort_values(["no"]).\
#                         groupby(["bacno", "cano"])["total_acc_min"]\
#                         .apply(lambda x: x.rolling(2).apply(lambda x: x[1] - x[0]).fillna(-1))
# bacno_cano_min_diff_latter = pd.DataFrame(bacno_cano_min_diff_latter).reset_index()
# bacno_cano_min_diff_latter.columns = ["no" ,"bacno_cano_min_diff_latter"]

# all_features = pd.merge(all_features, bacno_cano_min_diff_latter, left_on="no", 
#                         right_on="no", how='left')
# all_features.head(3)

#### (特徵) 最後14次的交易時間差平均

In [62]:
# 目前看起來只是交易時間互減而已
# bacno_cano_no_14_loctm_acc_sec_diff = all_features[["no", "bacno", "cano", "total_acc_sec"]].sort_values(["no"]).\
#                         groupby(["bacno", "cano"])["total_acc_sec"].apply(lambda x: x[-15:].diff().fillna(0).mean())

# bacno_cano_no_14_loctm_acc_sec_diff = pd.DataFrame(bacno_cano_no_14_loctm_acc_sec_diff).reset_index()
# bacno_cano_no_14_loctm_acc_sec_diff.columns = ["bacno", "cano", "bacno_cano_no_14_loctm_acc_sec_diff"]

# all_features = pd.merge(all_features, bacno_cano_no_14_loctm_acc_sec_diff, left_on=['bacno', 'cano'], 
#                         right_on=['bacno', 'cano'], how='left')
# all_features.head(3)

,index,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,...,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_all_conam_pos_trend,bacno_cano_no_14_conam_pos_trend,bacno_cano_no_recent_14_conam_pos_trend,bacno_cano_sec_diff_latter,bacno_cano_min_diff_latter,bacno_cano_no_14_loctm_acc_sec_diff
0,0,6881,113261,38038,513.80,0,0,0,0,0,...,11.0,0.0,0,0.0,-8.237000,-7.243333,-7.243333,39.0,0.65,28457.800000
1,1,0,134508,45725,465.62,0,0,0,0,0,...,2.0,1.0,1,1.0,-3.120000,-45.576000,-45.576000,65145.0,1085.75,372224.000000
2,2,6881,15408,188328,513.80,0,0,0,0,0,...,1.0,0.0,0,0.0,10.227222,-13.696667,-13.696667,0.0,0.00,435270.466667


#### (特徵) 最近5次的交易時間差平均

In [21]:
# 目前看起來只是交易時間互減而已
bacno_cano_recent_no_5_loctm_acc_min_diff = all_features[["no", "bacno", "cano", "total_acc_min"]].sort_values(["no"]).\
                        groupby(["bacno", "cano"])["total_acc_min"].apply(lambda x: x.diff().rolling(5, min_periods=1).mean()).fillna(0)

bacno_cano_recent_no_5_loctm_acc_min_diff = pd.DataFrame(bacno_cano_recent_no_5_loctm_acc_min_diff).reset_index()
bacno_cano_recent_no_5_loctm_acc_min_diff.columns = ["no", "bacno_cano_recent_no_5_loctm_acc_min_diff"]

all_features = pd.merge(all_features, bacno_cano_recent_no_5_loctm_acc_min_diff, left_on=["no"], 
                        right_on=["no"], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_7_times_conam_max_by_no,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_max_by_no,bacno_cano_conam_cumsum_mean,bacno_cano_no_mean_loctm_acc_min_diff,bacno_cano_recent_no_5_loctm_acc_min_diff
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1,1,0,0,2385.75,1458.33,2385.75,561.301096,1620.892616,189.563333
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1,0,0,0,1881.93,812.26,1881.93,355.962083,5288.661364,967.920000
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1,1,0,0,776.57,683.25,1304.96,552.063333,6931.104902,1939.836667


#### (特徵) 該帳戶的交易時間差平均

In [20]:
bacno_cano_no_mean_loctm_acc_min_diff = all_features[["no", "bacno", "cano", "total_acc_min"]].sort_values("no").\
                        groupby(["bacno", "cano"])["total_acc_min"].apply(lambda x: x.diff().mean()).fillna(0)

bacno_cano_no_mean_loctm_acc_min_diff = pd.DataFrame(bacno_cano_no_mean_loctm_acc_min_diff).reset_index()
bacno_cano_no_mean_loctm_acc_min_diff.columns = ["bacno", "cano", "bacno_cano_no_mean_loctm_acc_min_diff"]

all_features = pd.merge(all_features, bacno_cano_no_mean_loctm_acc_min_diff, left_on=["bacno", "cano"], 
                        right_on=["bacno", "cano"], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_7_times_conam_max_by_no,bacno_cano_conam_rolling_14_window_max,bacno_cano_14_times_conam_max_by_no,bacno_cano_conam_cumsum_mean,bacno_cano_no_mean_loctm_acc_min_diff
0,6881,113261,38038,513.80,5,0,0,0,0,0,...,1854121,1,1,0,0,2385.75,1458.33,2385.75,561.301096,1620.892616
1,0,134508,45725,465.62,5,0,0,2,0,0,...,1854121,1,0,0,0,1881.93,812.26,1881.93,355.962083,5288.661364
2,6881,15408,188328,513.80,5,0,0,0,0,0,...,1854121,1,1,0,0,776.57,683.25,1304.96,552.063333,6931.104902


#### (特徵) 每個帳號&卡號每筆交易紀錄前後一筆時間差取小

In [ ]:
all_features[["locdt", "loctm", "bacno", "cano"]].sort_values(["locdt", "loctm"]).\
                        groupby(["bacno", "cano"])["loctm"].apply(lambda x: x[])

In [49]:
all_features[all_features.index == 421635]
all_features[all_features["no"] == 421635]
# [:-len(test_pre)].index.unique().shape

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,...,flbmk_value_counts,hcefg_value_counts,csmcu_value_counts,flg_3dsmk_value_counts,prime_time_normal,prime_time_fraud,sleep_time,sleep_time_conam_lower_3000,bacno_cano_no_mean_loctm_acc_min_diff,bacno_cano_recent_no_5_loctm_acc_min_diff
576802,576802,6231,87970,112003,866.52,5,62,0,4,0,...,1932259,1799525,1585893,1854121,1,1,0,0,12631.441667,30.666667


#### (特徵) (重要) 統計每個帳號每日每小時出現次數 (少了直接爆降0.2)

In [75]:
bacno_cnt_by_locdt_loctm_hour = pd.DataFrame(all_features[["locdt", "loctm_hour", "bacno"]].\
                                 groupby(["locdt", "loctm_hour", "bacno"]).size()).reset_index()
bacno_cnt_by_locdt_loctm_hour.columns = ["locdt", "loctm_hour", "bacno", "bacno_cnt_by_locdt_loctm_hour"]
all_features = pd.merge(all_features, bacno_cnt_by_locdt_loctm_hour, left_on=["locdt", "loctm_hour", "bacno"],
                        right_on=["locdt", "loctm_hour", "bacno"], how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,bacno_cano_conam_rolling_7_window_sum,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_recent_7_conam_pos_trend,bacno_cano_sec_diff_latter,bacno_cnt_by_locdt_loctm_hour
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,3673.51,524.787143,601.579286,12.0,1.0,0,1.0,4.060244e-14,9575.0,4
1,0,134508,45725,465.62,0,0,0,0,0,9,...,2482.74,354.677143,383.248571,14.0,0.0,1,0.0,-5.442000e+01,4966.0,1
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,3892.49,556.070000,568.570000,10.0,0.0,0,0.0,-1.809000e+01,76914.0,1


In [117]:
## 一個小時內出現大量帳戶刷卡紀錄盜刷機率較高
# print(train_pre[["locdt", "loctm_hour", "bacno", "fraud_ind"]].groupby(["locdt", "loctm_hour", "bacno"]).mean().sort_values("fraud_ind", ascending=False))
# print(all_features[["locdt", "loctm_hour", "bacno"]].groupby(["locdt", "loctm_hour", "bacno"]).size().sort_values(ascending=False))
haha = train_pre[["locdt", "loctm_hour", "bacno", "fraud_ind"]].groupby(["locdt", "loctm_hour", "bacno"]).mean().sort_values("fraud_ind", ascending=False)
haha = haha.reset_index()
haha[haha["bacno"].eq(156870)]

,locdt,loctm_hour,bacno,fraud_ind
3466,33,3,156870,1.0
3490,33,9,156870,1.0
3599,33,2,156870,1.0
17152,61,14,156870,0.0
21670,60,14,156870,0.0
22625,60,15,156870,0.0
41466,61,15,156870,0.0
109653,66,11,156870,0.0
232521,46,14,156870,0.0
287540,56,15,156870,0.0


#### (特徵) 該帳戶及卡號該筆刷卡前遇到交易金額為0的消費紀錄次數

In [76]:
# all_features.rename(columns={"canam_is_0": "conam_is_0"}, inplace=True)
conam_is_0_cnt_by_bacno = all_features[["locdt", "bacno", "conam_is_0"]].sort_values("locdt").\
                                    groupby(["bacno"])["conam_is_0"].apply(lambda x: x[:-1].sum())

conam_is_0_cnt_by_bacno = pd.DataFrame(conam_is_0_cnt_by_bacno).reset_index()
conam_is_0_cnt_by_bacno.columns = ["bacno", "conam_is_0_cnt_by_bacno"]

all_features = pd.merge(all_features, conam_is_0_cnt_by_bacno, left_on="bacno", right_on="bacno", how='left')
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,ecfg,flbmk,flg_3dsmk,insfg,locdt,...,bacno_cano_conam_rolling_7_window_mean,bacno_cano_conam_rolling_14_window_mean,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_14_times_sleep_time_cnt,bacno_cano_7_sleep_time_conam_lower_3000_cnt,bacno_cano_14_sleep_time_conam_lower_3000_cnt,bacno_cano_no_recent_7_conam_pos_trend,bacno_cano_sec_diff_latter,bacno_cnt_by_locdt_loctm_hour,conam_is_0_cnt_by_bacno
0,6881,113261,38038,513.80,0,0,0,0,0,33,...,524.787143,601.579286,12.0,1.0,0,1.0,4.060244e-14,9575.0,4,5
1,0,134508,45725,465.62,0,0,0,0,0,9,...,354.677143,383.248571,14.0,0.0,1,0.0,-5.442000e+01,4966.0,1,0
2,6881,15408,188328,513.80,0,0,0,0,0,6,...,556.070000,568.570000,10.0,0.0,0,0.0,-1.809000e+01,76914.0,1,0


#### 一個帳號是否可擁有多個卡號

In [80]:
# all_features[["bacno", "cano"]].groupby("bacno").count()
print(train_pre[["bacno", "cano", "conam", "fraud_ind"]].groupby("bacno").agg({"cano": "count", "conam": "max", "fraud_ind": "mean"}).sort_values("fraud_ind", ascending=False))
## 卡號唯一值
print(all_features[["bacno", "cano"]].groupby("bacno").agg({"cano": lambda x: x.unique().shape[0]}))

        cano    conam  fraud_ind
bacno                           
35457      1  2156.97        1.0
121174     1  1016.34        1.0
146425     2  1174.97        1.0
157731     4   642.23        1.0
79233      2  1514.50        1.0
91084      2  2020.88        1.0
79805      1   708.98        1.0
73490      1  2013.19        1.0
15240      6   796.90        1.0
141498     1   695.22        1.0
26729      1  1440.67        1.0
19804      1   814.14        1.0
114790     2  1751.07        1.0
6794       2  2109.26        1.0
103101     1     0.00        1.0
115828     1     0.00        1.0
97215      1  1686.40        1.0
3179       1  2020.01        1.0
40669      3  1912.68        1.0
5110       2    84.37        1.0
99340      1   420.41        1.0
113697     1   407.87        1.0
52778      1   103.45        1.0
103403     3   794.93        1.0
118773     1   831.73        1.0
44940      1  2142.88        1.0
3271       1  2519.72        1.0
4905       4   943.12        1.0
105068    

#### 特徵交叉項 (前幾名特徵相乘、相加、相除)
TODO: loctm value_cnt的特徵如法炮製 and cnt當類別去跟金額等其他連續變數做groupby

In [10]:
# all_features["contp_etymd"] = all_features["contp"].apply(str) + "_" + all_features["etymd"].apply(str)
# mapping
# mapping_contp_etymd = {}
# for idx, level in enumerate(all_features["contp_etymd"].unique()):
#     mapping_contp_etymd[level] = idx
# all_features["contp_etymd"] = all_features["contp_etymd"].replace(mapping_contp_etymd)

In [3]:
all_features.columns

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'loctm',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'no', 'loctm_hour',
       'loctm_min', 'loctm_sec', 'total_acc_sec', 'total_acc_min',
       'total_acc_hour', 'conam_is_0', 'cocam_u_cnt_by_acqic',
       'bacno_u_cnt_by_acqic', 'conam_level', 'loctm_value_counts',
       'loctm_hour_value_counts', 'csmcu_cnt_by_stocn', 'csmcu_cnt_by_scity',
       'csmcu_ucnt_by_bacno', 'bacno_value_counts', 'cano_value_counts',
       'mchno_value_counts', 'acqic_value_counts', 'mcc_value_counts',
       'ecfg_value_counts', 'insfg_value_counts', 'contp_value_counts',
       'etymd_value_counts', 'stocn_value_counts', 'scity_value_counts',
       'stscd_value_counts', 'ovrlt_value_counts', 'flbmk_value_counts',
       'hcefg_value_counts', 'csmcu_value_counts', 'flg_3dsmk_value_counts',
       'prime_time_normal', 'prime_time_fraud', 'sleep_

### 刪除locdt, loctm, no等相關unique變數

In [78]:
# all_features.drop("index",axis=1, inplace=True)
all_features.drop("locdt",axis=1, inplace=True)
all_features.drop("total_acc_sec",axis=1, inplace=True)
all_features.drop("total_acc_min",axis=1, inplace=True)
all_features.drop("total_acc_hour",axis=1, inplace=True)
all_features.drop("no",axis=1, inplace=True)

In [79]:
all_features.columns.tolist()

['acqic',
 'bacno',
 'cano',
 'conam',
 'csmcu',
 'ecfg',
 'flbmk',
 'flg_3dsmk',
 'insfg',
 'loctm',
 'mcc',
 'mchno',
 'ovrlt',
 'scity',
 'stocn',
 'loctm_hour',
 'loctm_min',
 'loctm_sec',
 'conam_is_0',
 'cocam_u_cnt_by_acqic',
 'bacno_u_cnt_by_acqic',
 'conam_level',
 'loctm_value_counts',
 'loctm_hour_value_counts',
 'csmcu_cnt_by_stocn',
 'csmcu_cnt_by_scity',
 'csmcu_ucnt_by_bacno',
 'bacno_value_counts',
 'cano_value_counts',
 'mchno_value_counts',
 'acqic_value_counts',
 'mcc_value_counts',
 'ecfg_value_counts',
 'insfg_value_counts',
 'contp_value_counts',
 'etymd_value_counts',
 'stocn_value_counts',
 'scity_value_counts',
 'stscd_value_counts',
 'ovrlt_value_counts',
 'flbmk_value_counts',
 'hcefg_value_counts',
 'csmcu_value_counts',
 'flg_3dsmk_value_counts',
 'prime_time_normal',
 'prime_time_fraud',
 'sleep_time',
 'sleep_time_conam_lower_3000',
 'contp_0',
 'contp_1',
 'contp_2',
 'contp_3',
 'contp_4',
 'contp_5',
 'contp_6',
 'etymd_0',
 'etymd_1',
 'etymd_2',
 'et

### 合併One-Hot轉Latent的特徵

In [80]:
one_hot_cols = ['ecfg', 'flbmk','flg_3dsmk', 'insfg', 'ovrlt', 'ecfg_value_counts', 'insfg_value_counts', 'ovrlt_value_counts', 'contp_0',
 'contp_1', 'contp_2', 'contp_3', 'contp_4','contp_5', 'contp_6', 'etymd_0', 'etymd_1', 'etymd_2', 'etymd_3', 'etymd_4',
 'etymd_5', 'etymd_6', 'etymd_7', 'etymd_8', 'etymd_9', 'etymd_10', 'iterm_0', 'iterm_1', 'iterm_2', 'iterm_3', 'iterm_4',
 'iterm_5', 'iterm_6', 'iterm_7', 'iterm_8', 'stscd_0', 'stscd_1', 'stscd_2', 'stscd_3', 'stscd_4', 'hcefg_0',
 'hcefg_1', 'hcefg_2', 'hcefg_3', 'hcefg_4', 'hcefg_5', 'hcefg_6', 'hcefg_7', 'hcefg_8', 'hcefg_9',
 'conam_mean_by_ovrlt', 'conam_mean_by_flg_3dsmk']

for col in one_hot_cols:
    del all_features[col]
all_features.shape

(1938653, 74)

In [81]:
one_hot_to_latent_features = pd.read_csv("one_hot_to_latent_features.csv", index_col=None)
one_hot_to_latent_features = one_hot_to_latent_features.apply(lambda x: round(x, 3))
# latent6 latent8 0太多
del one_hot_to_latent_features["latent_6"]
del one_hot_to_latent_features["latent_8"]
one_hot_to_latent_features.head()

,latent_0,latent_1,latent_2,latent_3,latent_4,latent_5,latent_7,latent_9,latent_10,latent_11,latent_12,latent_13,latent_14,latent_15,latent_16
0,0.307,0.421,0.411,0.149,0.125,0.124,0.294,0.155,0.443,0.222,0.000,0.351,0.0,0.000,0.000
1,0.000,0.602,0.713,0.716,0.909,0.047,0.236,0.218,0.493,0.282,0.355,0.678,0.0,0.093,0.000
2,0.307,0.421,0.411,0.149,0.125,0.124,0.294,0.155,0.443,0.222,0.000,0.351,0.0,0.000,0.000
3,0.491,0.088,0.000,0.473,0.038,0.467,0.314,0.072,0.171,0.374,0.045,0.000,0.0,0.000,0.030
4,0.313,0.647,0.493,0.329,0.259,0.000,0.219,0.262,0.000,0.839,0.000,0.000,0.0,0.000,0.083


In [82]:
all_features = pd.concat([all_features, one_hot_to_latent_features], axis=1)
all_features.head(3)

,acqic,bacno,cano,conam,csmcu,loctm,mcc,mchno,scity,stocn,...,latent_5,latent_7,latent_9,latent_10,latent_11,latent_12,latent_13,latent_14,latent_15,latent_16
0,6881,113261,38038,513.80,0,172652,457,59333,0,102,...,0.124,0.294,0.155,0.443,0.222,0.000,0.351,0.0,0.000,0.0
1,0,134508,45725,465.62,0,105114,451,0,5817,102,...,0.047,0.236,0.218,0.493,0.282,0.355,0.678,0.0,0.093,0.0
2,6881,15408,188328,513.80,0,152458,457,59333,0,102,...,0.124,0.294,0.155,0.443,0.222,0.000,0.351,0.0,0.000,0.0


In [83]:
for col in all_features:
    if type(all_features[col].values[0]) == np.float64:
        all_features[col] = np.round(all_features[col], 3)
# all_features["bacno_cano_no_recent_14_conam_pos_trend"].head(3)

### 輸出檔案

In [17]:
train_pre_2 = all_features[:len(train_pre)]
test_pre_2 = all_features[len(train_pre):]

print(train_pre_2.shape)
print(test_pre_2.shape)

train_pre_2 = pd.concat([train_pre_2, train_pre["fraud_ind"]], axis=1)

print(train_pre_2.drop_duplicates().reset_index(drop=True).shape)
print(train_pre_2.shape)

(1516988, 62)
(421665, 62)
(1516988, 63)
(1516988, 63)


In [ ]:
all_features.columns

In [23]:
all_features[['bacno_cano_7_times_conam_max_by_no',
              'bacno_cano_conam_rolling_14_window_max',
              'bacno_cano_14_times_conam_max_by_no',
              'bacno_cano_conam_cumsum_mean',
             "bacno_cano_recent_no_5_loctm_acc_min_diff",
             "bacno_cano_no_mean_loctm_acc_min_diff"]].to_csv("all_features_18_other_features_2.csv")

In [19]:
all_features.to_csv("all_features_18_other_features_2.csv")

#### 存一個pickle檔紀錄欄位

In [85]:
file_name = "train_pre_18_fix_index"
# train_file = pd.read_csv(file_name + ".csv")
colname_record_file = pd.read_pickle("colname_record_file.pickle")
colname_record_file[file_name] = train_pre_2.columns.tolist()
pickle.dump(colname_record_file, open('colname_record_file.pickle', 'wb'))

In [30]:
pd.read_pickle("colname_record_file.pickle")

{'train_pre_7': ['acqic',
  'bacno',
  'cano',
  'conam',
  'contp',
  'csmcu',
  'ecfg',
  'etymd',
  'flbmk',
  'flg_3dsmk',
  'hcefg',
  'insfg',
  'iterm',
  'loctm',
  'mcc',
  'mchno',
  'ovrlt',
  'scity',
  'stocn',
  'stscd',
  'loctm_hour',
  'loctm_min',
  'loctm_sec',
  'conam_level',
  'loctm_value_counts',
  'loctm_hour_value_counts',
  'csmcu_cnt_by_stocn',
  'csmcu_cnt_by_scity',
  'bacno_value_counts',
  'cano_value_counts',
  'mchno_value_counts',
  'acqic_value_counts',
  'mcc_value_counts',
  'ecfg_value_counts',
  'insfg_value_counts',
  'contp_value_counts',
  'etymd_value_counts',
  'stocn_value_counts',
  'scity_value_counts',
  'stscd_value_counts',
  'ovrlt_value_counts',
  'flbmk_value_counts',
  'hcefg_value_counts',
  'csmcu_value_counts',
  'flg_3dsmk_value_counts',
  'conam_mean_by_mchno',
  'conam_mean_by_acqic',
  'conam_mean_by_mcc',
  'conam_mean_by_etymd',
  'conam_mean_by_locdt',
  'conam_mean_by_loctm_hour',
  'conam_mean_by_loctm_min',
  'conam_me

In [86]:
train_pre_2.to_csv("train_pre_18_fix_index.csv", index=None)
test_pre_2.to_csv("test_pre_18_fix_index.csv", index=None)

#### 同一個level授權金額或授權時間是否有outlier/ 數量是多少

#### 同一個level授權金額或授權時間是否有outlier